In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import datetime

In [2]:
path = '/Users/ianndelap/code/ianndelap/market_sentiment/raw_data/dump.json'
df = pd.read_json(path, nrows=1000, lines=True)
df_new = df[['full_text','favorite_count','symbols','created_at','lang','favorite_count','hashtags','retweet_count',]]

In [3]:
def tickerintc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'intc' in newlist:
            return 1
        else:
            return 0
def tickerbtc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'btc' in newlist:
            return 1
        else:
            return 0


In [4]:
def hashtag_intc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        intc = ['intel','intc']
        if intc[0] in newlist or intc[1] in newlist:
            return 1
        else:
            return 0

# hashtags and ticker columns 

In [7]:
df_new['#intc'] = df_new['symbols'].apply(hashtag_intc)        

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_50804/230576605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['#intc'] = df_new['symbols'].apply(hashtag_intc)


In [10]:
df_new['intc_ticker'] = df_new['symbols'].apply(tickerintc)        
#df_new['bynd_ticker'] = df_new['symbols'].apply(tickerbynd)        
#df_new['ge_ticker'] = df_new['symbols'].apply(tickerge)        
df_new['btc_ticker'] = df_new['symbols'].apply(tickerbtc)

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_50804/1183199101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['intc_ticker'] = df_new['symbols'].apply(tickerintc)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_50804/1183199101.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['btc_ticker'] = df_new['symbols'].apply(tickerbtc)


In [11]:
df_new

,full_text,favorite_count,symbols,created_at,lang,favorite_count,hashtags,retweet_count,#intc,intc_ticker,btc_ticker
0,Расцвет человеческого общества зависит от двух...,2,"[btc, alts, wan, strat, mtl, nano, nkn, kmd, z...",2020-03-08 03:19:20+00:00,ru,2,NaN,0,0.0,0.0,1.0
1,I’m waiting for the volatility to settle with ...,1,[btc],2020-03-08 03:20:37+00:00,en,1,NaN,0,0.0,0.0,1.0
2,Buy $BTC this is not a fucking drill,2,[BTC],2020-03-08 03:17:32+00:00,en,2,NaN,0,0.0,0.0,1.0
3,"$btc $btcusd nvm, 8,8k target hit",1,"[btc, btcusd]",2020-03-08 03:17:50+00:00,en,1,NaN,0,0.0,0.0,1.0
4,"Mar 8, 2020 03:12:00 UTC The price of $BTC cur...",0,[BTC],2020-03-08 03:13:02+00:00,en,0,NaN,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
995,14時51分 \n仮想通貨取引所Binance出資の SafePal ハードウォレットが発売...,0,"[BTC, FLOW, SFP]",2021-04-14 05:51:10+00:00,ja,0,NaN,0,0.0,0.0,1.0
996,He posts paid recommendations for free! He has...,0,"[mana, nexo, zil, klay, ele, btcb, eth, vet, x...",2021-04-14 05:51:43+00:00,en,0,NaN,0,0.0,0.0,1.0
997,Unusual #volume spike in $FTTBTC\n\n17 times t...,2,"[FTTBTC, FTT, BTC]",2021-04-14 05:52:02+00:00,en,2,[volume],0,0.0,0.0,1.0
998,ang sarap niyoooo $ETH $BTC,1,"[ETH, BTC]",2021-04-14 05:51:33+00:00,tl,1,NaN,0,0.0,0.0,1.0


#cleaning text

In [12]:
import re
def remove_urls(text):
    text = re.sub(r'https?://\S+', '', text)
    return text

# filtering per ticker

In [13]:
intel = df_new[df_new['intc_ticker'] == 1.0]
#intel = df_new[df_new['#intc'] == 1.0]

In [14]:
#changing by date 
intel['created_at'] = intel['created_at'].dt.date
intel = intel.sort_values('created_at', ascending=True)

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_50804/1047052366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intel['created_at'] = intel['created_at'].dt.date


In [16]:
len(intel)

44

In [10]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
text_result = embed(intel['full_text'])
vectordf = pd.DataFrame(text_result.numpy())

2021-09-02 11:20:01.837419: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-02 11:20:06.565005: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [11]:
#date_vec = pd.concat([vectordf,dfintel],axis = 1)

In [12]:
intel = intel.reset_index(drop=True)

In [13]:
intelvect = pd.concat([intel, vectordf], axis=1, join="inner")

In [14]:
intelvect

,full_text,favorite_count,symbols,created_at,lang,favorite_count,hashtags,retweet_count,#intc,intc_ticker,...,502,503,504,505,506,507,508,509,510,511
0,$FDX $D $INTC $T $GOOGL $GOOG Drone project a...,1,"[FDX, D, INTC, T, GOOGL, GOOG]",2018-05-09,en,1,NaN,0,1.0,1.0,...,-0.022217,0.019079,0.017394,-0.013539,0.071215,-0.012798,0.040813,-0.007588,-0.032403,0.010428
1,$INTC $GOOGL $FDX ...,0,"[INTC, GOOGL, FDX]",2018-05-09,en,0,NaN,0,1.0,1.0,...,0.006516,-0.006242,0.034683,-0.093543,0.031500,-0.029494,0.076364,-0.018019,-0.041867,0.069087
2,#estate asset watch - price / volume leaders -...,0,"[AMZN, AAPL, WMT, BABA, FB, NVDA, MU, DIS, GOO...",2018-05-09,en,0,[estate],0,1.0,1.0,...,0.028275,0.019678,-0.053702,-0.078718,0.047834,-0.023283,0.068030,-0.029323,-0.016944,-0.007516
3,Retweeted AS Tech (@AlphSt_Tech):\n\nZTE’s mai...,0,"[QCOM, INTC, AVGO, ACIA]",2018-05-09,en,0,"[ZTE, smartphones, china]",0,1.0,1.0,...,-0.018705,-0.039034,-0.041419,0.024290,0.084653,0.022990,-0.059361,0.021477,-0.028428,0.002707
4,Intel’s A.I. Director Singer Lays Out the Visi...,9,"[INTC, NVDA, AMD, GOOGL]",2018-05-09,en,9,NaN,3,1.0,1.0,...,-0.033510,-0.015773,-0.053097,-0.027476,0.019757,-0.014577,-0.041532,0.031148,-0.011246,-0.033173
5,Price Moves vs Expected Moves $MU $AMD $GOOGL ...,0,"[MU, AMD, GOOGL, NVDA, FB, TWTR, INTC, MSFT, J...",2018-05-09,en,0,NaN,0,1.0,1.0,...,0.011145,-0.017000,-0.051541,-0.092196,0.044260,-0.038817,0.083346,-0.013085,-0.012482,0.008286
6,$CAVM $QCOM $SSNLF $INTC https://t.co/TkIE1YxAyZ,0,"[CAVM, QCOM, SSNLF, INTC]",2018-05-09,ro,0,NaN,0,1.0,1.0,...,0.050605,-0.006404,0.013065,-0.125844,-0.003951,0.016810,0.021331,-0.048500,-0.050407,0.051700
7,The short sell volume percent for $INTC on 201...,0,"[INTC, NVDA, QCOM, TXN, AMD]",2018-05-09,en,0,NaN,0,1.0,1.0,...,-0.038844,-0.073154,-0.031251,-0.050175,0.023398,0.048878,-0.033747,-0.082363,0.030996,-0.029770
8,"$INTC Valuation, profitability and cash flow s...",0,[INTC],2018-05-09,en,0,NaN,0,1.0,1.0,...,0.020770,-0.069230,-0.016965,-0.062308,0.007944,0.084608,-0.013602,-0.038253,-0.044361,0.006757
9,Parthenon Increases Position in Hudson Technol...,0,"[HDSN, INTC]",2018-05-09,en,0,NaN,0,1.0,1.0,...,0.019062,0.005120,-0.047725,-0.074274,0.058666,-0.013635,0.022706,-0.002783,-0.038635,-0.056130


In [15]:
intelvect.columns

Index([     'full_text', 'favorite_count',        'symbols',     'created_at',
                 'lang', 'favorite_count',       'hashtags',  'retweet_count',
                '#intc',    'intc_ticker',
       ...
                    502,              503,              504,              505,
                    506,              507,              508,              509,
                    510,              511],
      dtype='object', length=522)

In [16]:
new_X = intelvect.drop(['favorite_count','intc_ticker','hashtags','retweet_count','#intc','full_text','lang','symbols'], axis = 1)


In [17]:
toselect = [n for n in range(512)] + ['created_at']
super_X = intelvect[toselect].groupby(['created_at']).mean()

In [18]:
super_X

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
created_at,,,,,,,,,,,,,,,,,,,,,
2018-05-09,0.002013,0.019751,-0.061112,-0.053452,0.014588,-0.007566,-0.046591,-0.004552,0.035964,0.009192,...,-0.002837,-0.021904,-0.029084,-0.052432,0.033671,0.005543,0.009080,-0.017092,-0.016512,-0.003491
2018-09-19,0.017767,0.010567,-0.041781,-0.032025,0.001455,-0.009198,-0.044121,-0.005527,0.031180,0.009369,...,-0.000134,-0.024490,-0.021335,-0.044067,0.011141,0.000471,0.024973,-0.030123,0.000324,0.013352
2019-10-25,-0.008557,-0.005810,-0.081025,-0.069472,0.032675,-0.000467,-0.019597,-0.023256,0.023273,0.011064,...,0.003430,-0.006441,-0.020768,-0.044235,0.048921,0.001102,0.030783,-0.024481,-0.017773,0.010041
2019-10-26,-0.003064,0.009893,-0.062664,-0.033145,0.023007,-0.000285,-0.018655,-0.018999,-0.003562,0.018601,...,-0.021516,-0.025065,-0.015368,-0.068392,0.043491,-0.034474,0.029047,-0.047927,-0.047097,0.014941
2020-03-07,-0.019304,-0.010105,-0.085096,-0.027520,-0.023854,-0.004243,-0.006676,-0.017453,-0.011070,-0.035685,...,-0.004835,0.045689,-0.022817,-0.078355,0.014799,0.036630,0.007342,-0.029422,-0.051835,0.047741
2020-03-08,-0.016147,0.057787,-0.044029,-0.034091,-0.034508,-0.045104,-0.061859,-0.006148,0.075059,-0.033800,...,-0.018886,-0.029830,-0.017618,-0.079838,0.048455,0.048882,-0.006820,-0.046096,-0.022073,0.005437
2020-07-19,-0.048547,0.053698,-0.076474,-0.084090,0.055640,0.037787,-0.017293,-0.035448,0.014829,-0.009440,...,0.000080,-0.065101,0.001611,-0.056376,0.064510,0.049775,-0.055975,-0.020028,0.033691,0.016948
2021-04-14,-0.009235,-0.045371,-0.063306,0.000493,-0.024804,-0.040684,-0.059389,-0.002495,0.048198,0.052132,...,0.041858,-0.004692,-0.050245,-0.082796,0.058784,-0.013084,0.062133,-0.051675,-0.009062,0.023085
2021-08-25,0.045879,0.063103,-0.087480,-0.045987,-0.015049,0.031918,-0.049873,-0.018007,0.011159,-0.071818,...,0.069344,-0.052967,-0.014123,-0.078606,-0.009865,0.012133,0.028318,-0.071641,0.039515,0.012662


In [19]:
newintel = new_X.groupby(['created_at']).mean()

In [20]:
newintel

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
created_at,,,,,,,,,,,,,,,,,,,,,
2018-05-09,0.002013,0.019751,-0.061112,-0.053452,0.014588,-0.007566,-0.046591,-0.004552,0.035964,0.009192,...,-0.002837,-0.021904,-0.029084,-0.052432,0.033671,0.005543,0.009080,-0.017092,-0.016512,-0.003491
2018-09-19,0.017767,0.010567,-0.041781,-0.032025,0.001455,-0.009198,-0.044121,-0.005527,0.031180,0.009369,...,-0.000134,-0.024490,-0.021335,-0.044067,0.011141,0.000471,0.024973,-0.030123,0.000324,0.013352
2019-10-25,-0.008557,-0.005810,-0.081025,-0.069472,0.032675,-0.000467,-0.019597,-0.023256,0.023273,0.011064,...,0.003430,-0.006441,-0.020768,-0.044235,0.048921,0.001102,0.030783,-0.024481,-0.017773,0.010041
2019-10-26,-0.003064,0.009893,-0.062664,-0.033145,0.023007,-0.000285,-0.018655,-0.018999,-0.003562,0.018601,...,-0.021516,-0.025065,-0.015368,-0.068392,0.043491,-0.034474,0.029047,-0.047927,-0.047097,0.014941
2020-03-07,-0.019304,-0.010105,-0.085096,-0.027520,-0.023854,-0.004243,-0.006676,-0.017453,-0.011070,-0.035685,...,-0.004835,0.045689,-0.022817,-0.078355,0.014799,0.036630,0.007342,-0.029422,-0.051835,0.047741
2020-03-08,-0.016147,0.057787,-0.044029,-0.034091,-0.034508,-0.045104,-0.061859,-0.006148,0.075059,-0.033800,...,-0.018886,-0.029830,-0.017618,-0.079838,0.048455,0.048882,-0.006820,-0.046096,-0.022073,0.005437
2020-07-19,-0.048547,0.053698,-0.076474,-0.084090,0.055640,0.037787,-0.017293,-0.035448,0.014829,-0.009440,...,0.000080,-0.065101,0.001611,-0.056376,0.064510,0.049775,-0.055975,-0.020028,0.033691,0.016948
2021-04-14,-0.009235,-0.045371,-0.063306,0.000493,-0.024804,-0.040684,-0.059389,-0.002495,0.048198,0.052132,...,0.041858,-0.004692,-0.050245,-0.082796,0.058784,-0.013084,0.062133,-0.051675,-0.009062,0.023085
2021-08-25,0.045879,0.063103,-0.087480,-0.045987,-0.015049,0.031918,-0.049873,-0.018007,0.011159,-0.071818,...,0.069344,-0.052967,-0.014123,-0.078606,-0.009865,0.012133,0.028318,-0.071641,0.039515,0.012662


# target 

In [21]:
import yfinance as yf

In [22]:
tickers = yf.download(tickers = 'INTC BYND GE BTC', interval='1d',start="2018-01-01", end="2021-08-01")

[*********************100%***********************]  4 of 4 completed


In [23]:
INTC = tickers['Close']['INTC']

In [24]:
def onezero(x):
    if x>0:
        return 1
    else:
        return 0

In [25]:
y_target = pd.Series(INTC).diff().apply(onezero)

In [30]:
intDF = pd.merge(newintel, y_target, left_index=True, right_index= True)

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,INTC
2018-05-09,0.002013,0.019751,-0.061112,-0.053452,0.014588,-0.007566,-0.046591,-0.004552,0.035964,0.009192,...,-0.021904,-0.029084,-0.052432,0.033671,0.005543,0.009080,-0.017092,-0.016512,-0.003491,1
2018-09-19,0.017767,0.010567,-0.041781,-0.032025,0.001455,-0.009198,-0.044121,-0.005527,0.031180,0.009369,...,-0.024490,-0.021335,-0.044067,0.011141,0.000471,0.024973,-0.030123,0.000324,0.013352,1
2019-10-25,-0.008557,-0.005810,-0.081025,-0.069472,0.032675,-0.000467,-0.019597,-0.023256,0.023273,0.011064,...,-0.006441,-0.020768,-0.044235,0.048921,0.001102,0.030783,-0.024481,-0.017773,0.010041,1
2021-04-14,-0.009235,-0.045371,-0.063306,0.000493,-0.024804,-0.040684,-0.059389,-0.002495,0.048198,0.052132,...,-0.004692,-0.050245,-0.082796,0.058784,-0.013084,0.062133,-0.051675,-0.009062,0.023085,0


# building the right model

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier


In [42]:
X = intDF.drop(columns=['INTC'])
y = intDF['INTC']

In [43]:
model = LinearRegression()

In [55]:
results = cross_validate(model, X, y, cv=2)
results

{'fit_time': array([0.01110387, 0.01795006]),
 'score_time': array([0.02341413, 0.01278806]),
 'test_score': array([ 0., -1.])}

In [56]:
results['test_score'].mean()

-0.5

In [ ]:
#scoring